In [ ]:
# Importing the relevant libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.optimize import minimize
 
# Importing data from MAS website
df=pd.read_excel("input/mas.xlsx")
df.info()
df.describe()
 
# Setting the Date column to be the index of the dataframe
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
 
# Setting 21 Oct 2022 as the base date
base_date = '2022-10-21'
indexed_df = df / df.loc[base_date]
indexed_df = indexed_df * 100
 
# Calculating log returns and dropping the rows with missing data
log_returns = indexed_df.apply(lambda x: np.log(x / x.shift(1)))
df = log_returns.dropna()
 
# Multiplying cross rates with minus one to convert FX returns into SGD as base currency
 
df1= df[['EUR','GBP','USD','AUD','CAD','CNY','HKD','INR','IDR','JPY','KRW','MYR','TWD','NZD','PHP','QAR','SAR','CHF','THB','AED','VND'
]]*-1
df1['NEER']=df['NEER']

## Optimization of NEER weights
We aim to estimate the weights of NEER using SG's trading partners. We first fit a regression equation with NEERS returns as the dependent variable (y) & cross rates returns with weights as independent variable (x). We seek to optimized the weights (ie beta) for a best fit regression line with lowest RMSE error.

In [1]:
# indicating dependent and independent variables
Y = df1['NEER'].values
X = df1[['EUR','GBP','USD','AUD','CNY','JPY','KRW','MYR','TWD']].values()
 
# objective function - sum of squared residuals
def objective_function(betas):
    Estimated_Y = np.dot(X, betas)
    return np.sum((Y - Estimated_Y) ** 2)
 
# Defining the constraint - sum of betas equals 1
constraints = {
    'type': 'eq',
    'fun': lambda betas: np.sum(betas) - 1
}
 
# Specifying the initial guess for betas - use any numbers that sum to 1
initial_betas = np.full(9, 1/9)
 
# Minimize the objective function and calculating the estimated values
result = minimize(objective_function, initial_betas, constraints=constraints)
 
optimized_betas = result.x
df1['Estimated_Y'] = np.dot(X, optimized_betas)
 
# The estimated weights on individual currencies are simply the optimized_betas multiplied with 100
Weights = [x*100 for x in optimized_betas]
 
# Assigning the corresponding FX names to the Weights and sorting in descending order
Currency=['EUR','GBP','USD','AUD','CNY','JPY','KRW','MYR','TWD']
 
Currency_Weights_dict = dict(zip(Currency, Weights))
 
sorted_Currency_Weights = sorted(Currency_Weights_dict.items(), key=lambda item: item[1], reverse=True)
 
# Plotting the estimated Weights
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
bars=plt.bar([p[0] for p in sorted_Currency_Weights], [p[1] for p in sorted_Currency_Weights])
 
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 1), ha='center', va='bottom')
 
plt.xlabel('Currency')
plt.ylabel('Weight (%)')
plt.title('Optimal NEER Weights')
plt.tight_layout()
 
plt.show()

NameError: name 'df1' is not defined

# Backtesting NEER with official figures

In [ ]:
# Calculating cumulative log returns for deriving the official NEER (NEERo) and estimated NEER (NEERe)
cumulative_log_returnsY = np.cumsum(df1['NEER'])
NEERo = 100 * np.exp(cumulative_log_returnsY)
 
cumulative_log_returnsYhat = np.cumsum(df1['Estimated_Y'])
NEERe = 100 * np.exp(cumulative_log_returnsYhat)
 
# Plotting offical vs. estimated NEER
plt.figure(figsize=(8, 4))
plt.plot(NEERo, label='Official NEER')
plt.plot(NEERe, label='Estimated NEER')
plt.title('NEER levels, 21 Oct 2022=100')
plt.legend()
plt.show()